In [1]:
import pandas as pd
import seaborn as srn
import statistics as sts

# Limpeza e Tratamento

In [2]:
#importar dados
dataset = pd.read_csv('Desktop/ml_project1_data.csv', sep=',')

In [3]:
#Aqui vamos dar uma olhada nos formatos das colunas
# e tamanho da base
dataset.info()

#dataset.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [4]:
#Vamos agrupar as contagens semelhantes em 3 campos. 
#Somar todas as Campanhas, pois uma pessoa pode ter comprado em mais de 1 campanha
dataset['AllCmp'] = dataset['AcceptedCmp1'] + dataset['AcceptedCmp2'] + dataset['AcceptedCmp3'] + dataset['AcceptedCmp4'] + dataset['AcceptedCmp5'] + dataset['Response']
#Somar todo o montante gasto em todas as categorias
dataset['AllSpent'] = dataset['MntFishProducts'] + dataset['MntMeatProducts'] + dataset['MntFruits'] + dataset['MntSweetProducts'] + dataset['MntGoldProds'] + dataset['MntWines'] 
#Somar todas as compras feitas em todas as categorias
dataset['AllPurchases'] = dataset['NumDealsPurchases'] + dataset['NumCatalogPurchases'] + dataset['NumStorePurchases'] + dataset['NumStorePurchases'] + dataset['NumStorePurchases'] + dataset['NumWebPurchases']

#visualizar
dataset.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,AllCmp,AllSpent,AllPurchases
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,0,0,0,0,3,11,1,1,1617,33
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,0,0,0,0,3,11,0,0,27,10
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,0,0,0,0,3,11,0,0,776,41
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,0,0,0,0,3,11,0,0,53,16
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,0,0,0,0,3,11,0,0,422,31


In [5]:
#Check Ids duplicados 
dataset[dataset.duplicated(['ID'],keep=False)]
#Vamos dar uma olhada se temos linhas vazias na base
dataset.isnull().sum()
#24 linhas vazias em Income, vamos substituir elas pela mediana
dataset['Income'].fillna(sts.median(dataset['Income']), inplace=True)
#Aproveitando que tinha problemas na coluna Income, começarei a verificar outliers por ela
dataset['Income'].describe()

mediana = sts.stdev(dataset['Income'])

inc_dist = pd.DataFrame(columns=['stdev','1stdev','2stdev','3stdev','4stdev','5stdev'])

inc_dist.at['0','stdev'] = (dataset['Income'].loc[(dataset['Income'] <= 0.5 * mediana)].count())/round(dataset['Income'].count(),2)*100
inc_dist.at['0','1stdev'] = (dataset['Income'].loc[(dataset['Income'] <= mediana)].count())/round(dataset['Income'].count(),2)*100
inc_dist.at['0','2stdev'] = dataset['Income'].loc[(dataset['Income'] > mediana) & (dataset['Income'] <= 2*mediana)].count()/round(dataset['Income'].count(),2)*100
inc_dist.at['0','3stdev'] = dataset['Income'].loc[(dataset['Income'] > 2*mediana) & (dataset['Income'] <= 3*mediana)].count()/round(dataset['Income'].count(),2)*100
inc_dist.at['0','4stdev'] = dataset['Income'].loc[(dataset['Income'] > 3*mediana) & (dataset['Income'] <= 4*mediana)].count()/round(dataset['Income'].count(),2)*100
inc_dist.at['0','5stdev'] = dataset['Income'].loc[(dataset['Income'] > 4*mediana) & (dataset['Income'] < 5*mediana)].count()/round(dataset['Income'].count(),2)*100

inc_dist

#Acima de 3stdev temos 15% da base, então vamos fazer o corte aqui
dataset = dataset.loc[dataset['Income'] < 3*mediana]
#agora temos o dataset sem os outliers >3stdev 
dataset #1905 rows



,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,AllCmp,AllSpent,AllPurchases
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,0,0,0,0,3,11,1,1,1617,33
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,0,0,0,0,3,11,0,0,27,10
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,0,0,0,0,3,11,0,0,776,41
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,0,0,0,0,3,11,0,0,53,16
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,0,0,0,0,3,11,0,0,422,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,2013-06-13,46,709,...,0,0,0,0,3,11,0,0,1341,26
2236,4001,1946,PhD,Together,64014.0,2,1,2014-06-10,56,406,...,0,1,0,0,3,11,0,1,444,32
2237,7270,1981,Graduation,Divorced,56981.0,0,0,2014-01-25,91,908,...,0,0,0,0,3,11,0,1,1241,45
2238,8235,1956,Master,Together,69245.0,0,1,2014-01-24,8,428,...,0,0,0,0,3,11,0,0,843,43


## Quantas pessoas participaram de x campanhas?

In [6]:
#Vamos dar uma olhada nas pessoas que compraram em cada campanha? 
df_3 = dataset[['ID','AllCmp']].groupby('AllCmp').count().reset_index()

df_3['Percent'] = (df_3['ID']/df_3['ID'].sum()).map('{:,.2%}'.format)

df_3
#A maioria das pessoas abordadas - 72% - não fez nenhuma compra nas campanhas

,AllCmp,ID,Percent
0,0,1501,78.79%
1,1,285,14.96%
2,2,96,5.04%
3,3,13,0.68%
4,4,9,0.47%
5,5,1,0.05%


## Quantas pessoas compraram em cada campanha?

In [7]:
agrupado = dataset[['AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','Response']].sum()
agrupado = agrupado/dataset['AllCmp'].sum()

agrupado_v2 = agrupado.to_frame()
agrupado_v2.rename(columns={0:'Percentage'},inplace=True)
agrupado_v2['Percentage'] = agrupado_v2['Percentage'].map('{:,.2%}'.format)
agrupado_v2

,Percentage
AcceptedCmp1,8.80%
AcceptedCmp2,2.87%
AcceptedCmp3,23.34%
AcceptedCmp4,19.93%
AcceptedCmp5,5.57%
Response,39.50%


## Quem reclamou comprou qual campanha?

In [8]:
#tirando todo mundo que nao reclamou
dataset2 = dataset.loc[dataset['Complain'] != 0]

#qual o percentagem de reclamantes na base total? 
dataset2['Complain'].sum()/dataset['Complain'].count()
#Cerca de 0.9% da base, podemos ignorar mas só por curiosidade, vamos checar a distribuicao dos reclamantes

#contagem
agrupado = dataset2[['AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','Response']].sum()
agrupado = agrupado/dataset2['AllCmp'].sum()

agrupado_v2 = agrupado.to_frame()
agrupado_v2.rename(columns={0:'Percentage'},inplace=True)
agrupado_v2['Percentage'] = agrupado_v2['Percentage'].map('{:,.2%}'.format)
agrupado_v2

,Percentage
AcceptedCmp1,0.00%
AcceptedCmp2,0.00%
AcceptedCmp3,33.33%
AcceptedCmp4,0.00%
AcceptedCmp5,0.00%
Response,66.67%


#maior parte das pessoas que reclamaram são do grupo Target, mas todo o grupo de reclamantes 
representa somente 0.9% da amostragem total


# Customer Behavior Segmentation

In [ ]:
# Idealmente a melhor maneira para realizar a analise de segmentação de clientes não objetivas,
# seria utilizando um método de clusterização, mas para mantermos esta analise simples, optarei por 
# utilizar um matriz RFM.


# Correlações

In [24]:
#Existe alguma correlação entre quem faz qualquer tipo de compra nas categorias de Purchase e
#estar no Target?

dataset['AllPurchases'].loc[dataset['AllPurchases'] > 0].corr(dataset['Response'])
#Correlação de 0.049

dataset['NumWebPurchases'].loc[dataset['NumWebPurchases'] > 0].corr(dataset['Response'])
#Correlação de 0.1126

dataset['NumStorePurchases'].loc[dataset['NumStorePurchases'] > 0].corr(dataset['Response'])
#Correlação de -0.0261

dataset['NumDealsPurchases'].loc[dataset['NumDealsPurchases'] > 0].corr(dataset['Response'])
#Correlação de 0.09457

dataset['NumCatalogPurchases'].loc[dataset['NumCatalogPurchases'] > 0].corr(dataset['Response'])
#Correlação de 0.0737

dataset.corr()

##AllCmp e Response 0.73
#Response e Cmp3 0.26

#AllSpent e Income 0.74
#AllPurchases e Income 0.71



,ID,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,AllCmp,AllSpent,AllPurchases
ID,1.000000,0.007939,-0.014496,0.002001,-0.009313,-0.056366,-0.027027,0.015168,-0.002316,-0.012231,...,-0.016463,-0.003029,0.000250,0.037071,NaN,NaN,-0.022786,-0.035457,-0.019495,-0.023825
Year_Birth,0.007939,1.000000,-0.264160,0.252124,-0.428903,-0.006723,-0.177609,-0.013355,-0.046948,-0.032819,...,0.005565,0.001835,-0.019798,-0.044446,NaN,NaN,-0.009065,-0.013190,-0.131766,-0.190496
Income,-0.014496,-0.264160,1.000000,-0.473413,0.270889,-0.005580,0.697198,0.463966,0.576771,0.466699,...,0.162702,0.178659,0.063370,-0.026168,NaN,NaN,0.027116,0.143824,0.748685,0.716726
Kidhome,0.002001,0.252124,-0.473413,1.000000,-0.136910,0.014840,-0.482601,-0.342751,-0.394923,-0.359530,...,-0.104489,-0.102723,-0.064610,0.031961,NaN,NaN,-0.009401,-0.102365,-0.529951,-0.483326
Teenhome,-0.009313,-0.428903,0.270889,-0.136910,1.000000,0.011237,0.142275,-0.085673,-0.139893,-0.129742,...,-0.086540,-0.039231,0.010963,-0.006225,NaN,NaN,-0.097149,-0.058245,0.021506,0.214036
Recency,-0.056366,-0.006723,-0.005580,0.014840,0.011237,1.000000,0.006478,-0.001742,0.011478,-0.006966,...,0.000536,-0.030373,-0.012656,0.014515,NaN,NaN,-0.215499,-0.127418,0.011680,0.004339
MntWines,-0.027027,-0.177609,0.697198,-0.482601,0.142275,0.006478,1.000000,0.354034,0.557142,0.349586,...,0.299886,0.260642,0.195195,-0.037377,NaN,NaN,0.144203,0.365088,0.898241,0.762377
MntFruits,0.015168,-0.013355,0.463966,-0.342751,-0.085673,-0.001742,0.354034,1.000000,0.550722,0.618112,...,0.089285,0.079703,-0.029103,-0.004889,NaN,NaN,0.055851,0.044025,0.594001,0.489262
MntMeatProducts,-0.002316,-0.046948,0.576771,-0.394923,-0.139893,0.011478,0.557142,0.550722,1.000000,0.606729,...,0.178646,0.227699,0.000348,-0.024089,NaN,NaN,0.137200,0.169443,0.827841,0.560290
MntFishProducts,-0.012231,-0.032819,0.466699,-0.359530,-0.129742,-0.006966,0.349586,0.618112,0.606729,1.000000,...,0.063139,0.127209,-0.017740,-0.015077,NaN,NaN,0.019810,0.020419,0.625455,0.478656
